In [2]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping 
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import StratifiedShuffleSplit
import utility
from gensim.models import KeyedVectors
from keras.models import load_model
import pickle

C:\Users\Shche\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Shche\Anaconda3\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Data

In [3]:
data =  pd.read_csv('Data/data_no_duplicates.csv', sep=',')
data

,Unnamed: 0,id,text,misogynous,misogyny_category,target
0,0,1,Please tell me why the bitch next to me in the...,1,dominance,active
1,1,3,"@abzdafab Dear cunt, please shut the fuck up.",1,dominance,active
2,2,4,RT @queenofdragonsb: Pls shut the fuck up bitch,1,dominance,active
3,3,5,"RT @21bIvck: ""when u gonna get your license"" S...",1,dominance,active
4,4,6,@SarahhWaqar @CallmeJaagii Bitch shut the fuck up,1,dominance,active
5,5,7,ok babies i'll go to sleep ok bitch shut the f...,1,dominance,passive
6,6,8,You are a fucking ugly bitch!shut the fuck up?...,1,dominance,active
7,7,9,RT @bnixole: bitch shut the fuck up you're fuc...,1,dominance,active
8,8,10,RT @vaintshit: shut the fuck up and come suck...,1,sexual_harassment,active
9,9,11,@AnisaJomha ice has more talent in his ejecula...,1,sexual_harassment,active


In [4]:
# preprocess text: to lower case, http instead of a full link, @ instead of a full @-mention
utility.text_preprocessing(data, 'text')

In [5]:
# split data into train and test sets
split = StratifiedShuffleSplit(test_size = 0.1, random_state = 42)
for train_index, test_index in split.split(data, data[['misogynous', 'misogyny_category', 'target']]):
    train = data.loc[train_index]
    test = data.loc[test_index]

# Misogyny

## Text preprocessing for NN

In [6]:
# lists of tweets and labels
texts = list(train['text'])  # list of text samples
labels = list(train['misogynous'])

texts_test = list(test['text'])  # list of text samples
labels_test = list(test['misogynous'])

In [7]:
# vectorize tweets, turn tweets to sequences 
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences_test = tokenizer.texts_to_sequences(texts_test)

In [8]:
# pad sequences to the length = 50
data_train = pad_sequences(sequences, maxlen=50)
data_test = pad_sequences(sequences_test, maxlen=50)

In [9]:
# convert labels to binary class matrix
labels = np_utils.to_categorical(np.asarray(labels))
labels_test = np_utils.to_categorical(np.asarray(labels_test))

In [10]:
# split the data into a training set and a validation set
indices = np.arange(data_train.shape[0])
np.random.shuffle(indices)
data_train = data_train[indices]
labels = labels[indices]
nb_validation_samples = int(0.1 * data_train.shape[0])

x_train = data_train[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data_train[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

## FastText with pretrained vectors

In [10]:
# load pretrained 300-dimensional word vectors
en_model = KeyedVectors.load_word2vec_format('Data/wiki.en.vec')

In [11]:
# get a list of all pretrained words 
words = []
for word in en_model.vocab:
    words.append(word)

In [12]:
# create an embedding matrix, where each row is a vectorized pretrained word
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    try:
        embedding_vector = en_model.get_vector(word)
    except:
        embedding_vector = None
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [13]:
# an embedding layer for a model
embedding_layer = Embedding(len(word_index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=50,
                            trainable=False)

In [14]:
# train a model
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.5, activation='relu'))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.5, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=512,
         callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           2351400   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 64)            93440     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 2,477,994
Trainable params: 126,594
Non-trainable params: 2,351,400
_________________________________________________________________
None


C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is 

C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is 

C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is 

C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
C:\Users\Shche\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is 

Train on 3238 samples, validate on 359 samples
Epoch 1/100
3238/3238 [==============================] - 3s 975us/step - loss: 0.6842 - acc: 0.5578 - val_loss: 0.6797 - val_acc: 0.5376
Epoch 2/100
3238/3238 [==============================] - 1s 383us/step - loss: 0.6682 - acc: 0.5788 - val_loss: 0.6523 - val_acc: 0.6880
Epoch 3/100
3238/3238 [==============================] - 1s 358us/step - loss: 0.6374 - acc: 0.6516 - val_loss: 0.5989 - val_acc: 0.7159
Epoch 4/100
3238/3238 [==============================] - 1s 352us/step - loss: 0.6179 - acc: 0.6723 - val_loss: 0.5942 - val_acc: 0.7103
Epoch 5/100
3238/3238 [==============================] - 1s 354us/step - loss: 0.5990 - acc: 0.6881 - val_loss: 0.5849 - val_acc: 0.6992
Epoch 6/100
3238/3238 [==============================] - 1s 356us/step - loss: 0.5968 - acc: 0.6810 - val_loss: 0.5776 - val_acc: 0.7103
Epoch 7/100
3238/3238 [==============================] - 1s 359us/step - loss: 0.5887 - acc: 0.6924 - val_loss: 0.5683 - val_acc: 0

In [15]:
# make predictions for the test set
y = model.predict(data_test)
y = y.argmax(axis=-1)
print("Accuracy:", metrics.accuracy_score(test['misogynous'], y))

Accuracy: 0.7825


In [16]:
# save the model and tokenizer
with open('Project Models/NN_tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f, pickle.HIGHEST_PROTOCOL)
model.save('Project Models/NN_model.h5')